## 0. Imports Preliminaries

In [1]:
#0. Preliminaries
import pandas as pd
import nltk
nltk.download('punkt')
from readability import Readability

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import string
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
from transformers import pipeline

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

c:\Users\maria\anaconda3\envs\fl_project_analysis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
model.safetensors:   6%|▌         | 31.5M/541M [00:16<04:26, 1.91MB/s]


KeyboardInterrupt: 

## 1. Load and Split Data

In [6]:
#1. Import Data
#chunked test from data_generation, needs to be redone 
df = pd.read_csv("../data/author_data.csv")

In [ ]:
df_train_val, df_test = train_test_split(df, train_size = 0.8, random_state = 42)
df_train, df_val = train_test_split(df_train_val, train_size = 0.8, random_state = 42)

## 2. Add Features

In [ ]:
#Flesch 
def flesch_readability_scale(text):
    r = Readability(text)
    f = r.flesch()
    return f.score

In [ ]:
# Sentiment Analysis (Positive Score)
def sentiment_analysis_score(text):
    results_senti = distilled_student_sentiment_classifier(text)
    positive_score = [x['score'] for x in results_senti if x['label'] == 'positive']
    score_out = positive_score[0] if len(positive_score) == 1 else np.nan
    return score_out                          

In [ ]:
#punctuation n-grams
punct_vectorizer = TfidfVectorizer(tokenizer=nltk.word_tokenize, use_idf=False, norm='l1', vocabulary=string.punctuation, ngram_range=(1, 3))

## 3. Apply Features to Datasets

In [ ]:
def preprocess_data(df, train = False):
    df['flesch_score'] = df['text'].apply(flesch_readability_scale)
    df['sent_score'] = df['text'].apply(sentiment_analysis_score)
    #train has to be run first - a catch statement for that
    try:
        punct_features = punct_vectorizer.fit_transform(df) if train else punct_vectorizer.transform(df)
    except Exception as e:
        print(f"Error: {e}")